## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-23-03-42-00 +0000,wsj,Gunmen Wreak Chaos in Mexican Coastal Retreat ...,https://www.wsj.com/world/americas/gunmen-wrea...
1,2026-02-23-03-17-04 +0000,nypost,Heartbreaking vigil for Lake Tahoe avalanche v...,https://nypost.com/2026/02/22/us-news/heartbre...
2,2026-02-23-03-17-03 +0000,bbc,"Who was El Mencho, Mexico's most wanted man?",https://www.bbc.com/news/articles/c1jkw18e19jo...
3,2026-02-23-03-16-13 +0000,nyt,Armed Man Shot Dead at Mar-a-Lago by Trump’s S...,https://www.nytimes.com/2026/02/22/us/north-ca...
4,2026-02-23-03-15-02 +0000,nyt,"When Faced With Claims of Racism, Trump Points...",https://www.nytimes.com/2026/02/22/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2491/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,36
286,iran,13
84,new,13
92,hockey,12
104,olympic,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
27,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...,94
3,2026-02-23-03-16-13 +0000,nyt,Armed Man Shot Dead at Mar-a-Lago by Trump’s S...,https://www.nytimes.com/2026/02/22/us/north-ca...,87
142,2026-02-22-14-09-27 +0000,cbc,"Armed man shot, killed after entering perimete...",https://www.cbc.ca/news/world/trump-mar-a-lago...,87
95,2026-02-22-20-31-32 +0000,nypost,Trump congratulates USA men’s hockey team in p...,https://nypost.com/2026/02/22/sports/trump-con...,84
191,2026-02-22-06-32-45 +0000,nypost,Trump reveals his ‘new hero’ Supreme Court jus...,https://nypost.com/2026/02/22/us-news/trump-ca...,79


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
27,94,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...
7,55,2026-02-23-03-09-26 +0000,nypost,"Family of Austin Tucker Martin, shotgun-wieldi...",https://nypost.com/2026/02/22/us-news/shotgun-...
67,52,2026-02-22-22-52-24 +0000,nypost,"White House trolls Canada, Justin Trudeau with...",https://nypost.com/2026/02/22/sports/white-hou...
74,46,2026-02-22-22-13-39 +0000,nypost,Panicked passengers run for lives at Guadalaja...,https://nypost.com/2026/02/22/world-news/panic...
42,42,2026-02-23-00-55-41 +0000,bbc,Mexico's most wanted drug lord 'El Mencho' kil...,https://www.bbc.com/news/articles/cy4wywnrdd8o...
196,35,2026-02-22-04-47-09 +0000,nypost,Iran president says Tehran won’t ‘bow’ as war ...,https://nypost.com/2026/02/21/world-news/iran-...
37,25,2026-02-23-01-19-49 +0000,nypost,Chaos at LAX after TSA PreCheck and Global Ent...,https://nypost.com/2026/02/22/us-news/lax-mayh...
63,24,2026-02-22-23-03-02 +0000,nypost,Love-struck suspected Moscow mole in NYC tells...,https://nypost.com/2026/02/22/us-news/suspecte...
138,22,2026-02-22-15-15-42 +0000,bbc,Brown urges police probe into whether Andrew u...,https://www.bbc.com/news/articles/cj0dme87l84o...
100,22,2026-02-22-20-16-34 +0000,nypost,Trump sends hospital ship to Greenland — after...,https://nypost.com/2026/02/22/us-news/trump-se...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
